In [1]:
import pandas as pd
import numpy as np
from datetime import time

# A timespan 

In [2]:
# Set of period
period_length = 5 # min
total_operation_hours = 24 # hours

T = [i for i in range(1, total_operation_hours * 60 // period_length + 1)]

# print("List of periods T =", T)

# The set of stations
P_S, num of wharves Station S has


C_w, capacity of wharf w 

In [3]:
# Station and wharves dataframe
station_and_wharves_df = pd.read_excel('data.xlsx', sheet_name='Station_and_Wharf')
station_and_wharves_df.head()

,Station,Abbrev. Name,Wharf No,Capacity
0,Circular Quay,CQ,1,2
1,Circular Quay,CQ,2,2
2,Circular Quay,CQ,3,2
3,Circular Quay,CQ,4,2
4,Circular Quay,CQ,5,2


In [4]:
# Set of station includinh abbraviation
S = station_and_wharves_df['Station'].tolist()
S_abbrev = station_and_wharves_df['Abbrev. Name'].tolist() # no need for this?

# Circular 2[B],4[A,B],5[A,B] for Service; 1[A,B],2[A],3[A,B] for wait
# Barangaroo 1[A,B],4[A,B],5[A,B],2[A,B] for service;

# For each station S, num of wharves it has,PS.
# Example: Circular quay
P_CQ = len(station_and_wharves_df[station_and_wharves_df['Station'] == 'Circular Quay']['Wharf No'].unique())
print(f'Num of wharves in Circular Quay: P_CQ = {P_CQ}')

# For each wharf w in the system,Cw its capacity
# Example: Capacity of wharf 1 at Circular quay
C_CQ1 = station_and_wharves_df[(station_and_wharves_df['Station'] == 'Circular Quay')&(station_and_wharves_df['Wharf No'] == 1)]['Capacity'].iloc()[0]
print(f'Capaciyu of wharf 1 in Circular Quay: C_CQ1 = {C_CQ1}')


Num of wharves in Circular Quay: P_CQ = 5
Capaciyu of wharf 1 in Circular Quay: C_CQ1 = 2


In [5]:
# Xi (S1, S2), travel time between S1 and S2
TT_df = pd.read_excel('data.xlsx', sheet_name='Wharf to wharf transit times')
TT_df.head()


,Route,From wharf:,To wharf:,Time underway
0,F2,Circular Quay,Mosman,00:15:00
1,F2,Mosman,Taronga Zoo,00:06:00
2,F2,Taronga Zoo,Circular Quay,00:12:00
3,F2,Taronga Zoo,Cremorne Point,00:02:00
4,F3,Circular Quay,Drummoyne,00:14:00


In [6]:
# Example: Circular quay to Barangaroo
Xi_CQtoBar_t = TT_df[(TT_df['From wharf:'] == 'Circular Quay')&(TT_df['To wharf:'] == 'Barangaroo')]['Time underway '].iloc()[0] # datetime.time # there is a space after the col name 'Time underway '
Xi_CQtoBar = Xi_CQtoBar_t.hour * 60 + Xi_CQtoBar_t.minute # transfer into int

print(f'Travel time between Circular quay and Barangaroo, the \u03be(CQ,Bar), is {Xi_CQtoBar} minutes')

Travel time between Circular quay and Barangaroo, the ξ(CQ,Bar), is 10 minutes


# The lines:
 Each line l has a route (i.e. a sequence of stations to be visited), headways (detailed in a different bullet point below), and a set of times when the first sailing of that line can begin. We denote by Rl the route of line l, where Rl is a vector containing the sequence of stations. Throughout the paper, we use the notation S ∈ Rl to describe that S is one element of the vector, i.e., that the line stops there.

In [7]:
# first sheet: weekday trips
weekday_trips = pd.read_excel('data.xlsx', sheet_name='Week-day all trips')

# Display the DataFrame to check if it's loaded correctly
weekday_trips.head()

,Route No,Line description,Direction,Origin,Terminus,Vessel,Depart Origin,Service/ or Positioning trip,Dep. berth,Min. operating speed,...,Arrival at terminus,Min. dwell at terminus,Barangaroo intermed stop (BIS) arr. time,BIS berth,BIS dep. time,BIS Berthing time,BIS Clearance time,BIS Operating margin,Unnamed: 22,dw time
0,F2,Zoo/Mos,Outbound,Circular Quay,Mosman,FF3,06:48:00,Service,4A or 4B,13 knots,...,07:03:00,00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:07:00
1,F2,Zoo/Mos,Outbound,Circular Quay,Mosman,EC2,07:18:00,Service,4A or 4B,13 knots,...,07:33:00,00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:07:00
2,F2,Zoo/Mos,Outbound,Circular Quay,Mosman,24M?,07:48:00,Service,4A or 4B,13 knots,...,08:03:00,00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,What does 24M mean?,00:07:00
3,F2,Zoo/Mos,Outbound,Circular Quay,Mosman,FF3,08:18:00,Service,4A or 4B,13 knots,...,08:33:00,00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:07:00
4,F2,Zoo,Outbound,Circular Quay,Taronga Zoo,FF2,09:20:00,Service,4A or 4B,13 knots,...,09:32:00,00:06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:10:00


In [8]:
R = []
# F2 Rl

# line 1 Circular Quay --> Mosman (Outbound)
R.append(['Circular Quay', 'Mosman'])
# line 2 Mosman --> Circular Quay (Inbound)
R.append(['Mosman','Circular Quay'])
# line 3 Circular Quay --> Taronga Zoo (Outbound)
R.append(['Circular Quay', 'Taronga Zoo'])
# line 4 Taronga Zoo --> Circular Quay (Inbound)
R.append(['Taronga Zoo','Circular Quay'])

In [9]:
# F3
# line 5 Circular Quay --> Barangaroo --> Parramatta (Outbound)
R.append(['Circular Quay','Barangaroo','Parramatta'])

# line 6 Parramatta --> Barangeroo --> Circular Quay (Inbound)
R.append(['Parramatta','Barangaroo','Circular Quay'])

In [10]:
# F4 
# line 7 Circular Quay --> Barangaroo --> Pyrmont Bay (Outbound)
R.append(['Circular Quay','Barangaroo','Pyrmont Bay'])

# line 8 Pyrmont Bay --> Barangaroo --> Circular Quay (Inbound)
R.append(['Pyrmont Bay','Barangaroo','Circular Quay'])

In [11]:
# F5
# line 9 Circular Quay --> Circular Quay (Loop)
R.append(['Circular Quay','Circular Quay'])

In [12]:
# F6
# line 10 Circular Quay --> Mosman (Outbound)
R.append(['Circular Quay','Mosman'])

# line 11 Mosman --> Circular Quay (Inbound)
R.append(['Mosman','Circular Quay'])

In [13]:
# F7
# line 12 Circular Quay --> Double Bay (Outbound)
R.append(['Circular Quay','Double Bay'])

# linev13 Double Bay --> Circular Quay (Inbound)
R.append(['Double Bay','Circular Quay'])

In [14]:
# F8
# line 14 Circular Quay --> Cockatoo Is (Outbound)
R.append(['Circular Quay','Cockatoo Is'])

# line 15 Cockatoo Is --> Circular Quay (Inbound)
R.append(['Cockatoo Is','Circular Quay'])

# line 16 Circular Quay --> Woolwich (Outbound)
R.append(['Circular Quay','Woolwich'])

# line 17 Woolwich --> Circular Quay (Inbound)
R.append(['Woolwich','Circular Quay'])

In [15]:
# F9
# line 18 Circular Quay --> Rose Bay (Outbound)
R.append(['Circular Quay','Rose Bay'])

# line 19 Rose Bay --> Circular Quay (Inbound)
R.append(['Rose Bay','Circular Quay'])

# line 20 Circular Quay --> Watsons Bay (Outbound)
R.append(['Circular Quay','Watsons Bay'])

# line 21 Watsons Bay --> Circular Quay (Inbound)
R.append(['Watsons Bay','Circular Quay'])

In [16]:
# F10
# line 22 Barangaroo --> Blackwattle Bay (Outbound)
R.append(['Barangaroo','Blackwattle Bay'])

# line 23 CBlackwattle Bay --> Barangaroo (Inbound)
R.append(['Blackwattle Bay','Barangaroo'])

In [17]:
linedf = pd.DataFrame(R, index=[f'{line+1}' for line in range(23)])

## test take R1
linedf.loc['1'].dropna().tolist()
linedf.columns = ['Station1','Station2','Station3']
linedf


# need revise 

,Station1,Station2,Station3
1,Circular Quay,Mosman,None
2,Mosman,Circular Quay,None
3,Circular Quay,Taronga Zoo,None
4,Taronga Zoo,Circular Quay,None
5,Circular Quay,Barangaroo,Parramatta
6,Parramatta,Barangaroo,Circular Quay
7,Circular Quay,Barangaroo,Pyrmont Bay
8,Pyrmont Bay,Barangaroo,Circular Quay
9,Circular Quay,Circular Quay,None
10,Circular Quay,Mosman,None


In [18]:
#  C(l,S) the set of wharves in S that can be used.
# Example: For Sation Circular Quay
C_l1_CQ = station_and_wharves_df[station_and_wharves_df['Station'] == 'Circular Quay']['Wharf No'].unique().tolist()
print(f'the wharves in Circular Quay that can be used are: {C_l1_CQ}')

## Question here: 假设所有的在circular quay 的 wharves 都能用吗，还是参照表格现有的给定的，例如[4A,4B]

the wharves in Circular Quay that can be used are: [1, 2, 3, 4, 5]


# dwelling time
Clearance + berthing + operating margin in combination --> buffer time

dw for all lines'd origin are 1 time period

F2, all lines are with dw of 2 time periods for the terminus

F3, only one line one sailling is 1 time pirod, all other sailings for lines are with dw pf 2 time periods for the terminus, 1 time period for the Barangaroo intermed stop (BIS)

F4, all lines are with dw of 2 time periods for the terminus, 1 time period for the Barangaroo intermed stop (BIS)

F5 all lines are with dw of 2 time periods for the terminus

F6 all lines are with dw of 2 time periods for the terminus

F7 all lines are with dw of 2 time periods for the terminus

F8 all lines are with dw of 2 time periods for the terminus

F9, about half of the sailings for line 19 are with dw of 2 time periods, all other sailings for lines are with dw of 3 time periods for the terminus --> Assume dewlling time for all lines in F9 is 3

F10 all lines are with dw of 2 time periods for the terminus


In [19]:
linedf[['dw1','dw2','dw3']] = None #placeholder for dwelling time at each station ar each line

# For all origin, dwelling time is 1 time period
linedf['dw1'] = 1

# F3 (L5,L6), 1 time period for BIS, 2 time period dw for terminus
linedf.loc['5','dw2'] = 1
linedf.loc['5','dw3'] = 2
linedf.loc['6','dw2'] = 1
linedf.loc['6','dw3'] = 2

# F4 (L7,L8), 1 time period for BIS, 2 time period dw for terminus
linedf.loc['7','dw2'] = 1
linedf.loc['7','dw3'] = 2
linedf.loc['8','dw2'] = 1
linedf.loc['8','dw3'] = 2

# For all other lines termines, 2 time period dw 
linedf['dw2'].fillna(2, inplace=True) # since only F3 F4 have BIS, other lines's terminus is station 2
linedf

,Station1,Station2,Station3,dw1,dw2,dw3
1,Circular Quay,Mosman,None,1,2,None
2,Mosman,Circular Quay,None,1,2,None
3,Circular Quay,Taronga Zoo,None,1,2,None
4,Taronga Zoo,Circular Quay,None,1,2,None
5,Circular Quay,Barangaroo,Parramatta,1,1,2
6,Parramatta,Barangaroo,Circular Quay,1,1,2
7,Circular Quay,Barangaroo,Pyrmont Bay,1,1,2
8,Pyrmont Bay,Barangaroo,Circular Quay,1,1,2
9,Circular Quay,Circular Quay,None,1,2,None
10,Circular Quay,Mosman,None,1,2,None


# Copy from Github below


In [21]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [22]:
data = {'numberOfFirstSailings': [8,3,4,2,6]}
dataFrame = pd.DataFrame(data)
dataFrame


# numberOfFirstSailings, set of D(l)

,numberOfFirstSailings
0,8
1,3
2,4
3,2
4,6


In [23]:
# declare Gurobi Model
model = gp.Model("Ferry_ILP")
model.setParam("LogToConsole", 0)

Restricted license - for non-production use only - expires 2025-11-24


In [24]:
## Constrain 1a: each line has exactly one initial departure.
numberOfFirstSailings = dataFrame['numberOfFirstSailings'].to_list()

numberOfLines = len(numberOfFirstSailings)
Lset = range(0, numberOfLines)
Dset = list(range(0, len(Lset)))

# declare variable
x = model.addVars(numberOfLines, max(numberOfFirstSailings), vtype=GRB.BINARY, name="x")

# add constrain
for l in Lset:
    Dset[l] = list(range(0, numberOfFirstSailings[l]))
    model.addConstr(gp.quicksum(x[l, d] for d in Dset[l]) == 1)

In [ ]:
## Constrain 1b: each sailing, its line has exactly one vessel assigned to that task 
# in the time period associated to that sailing in the chosen departure of the line, 
# and that the same line has no vessel assigned in other time periods.

# Vset: Set of Vessels
# Lset: Set of Lines
# Dset: Set of Departures Times
# Zset: Set of Sailing Times
y = model.addVars(len(Vset), len(Lset), max(len(numberOfFirstSailings), vt=GRB.BINARY, name="y"))

# add constrain
for s in Zset:
    for d in Dset[l[s]]:
        model.addConstr(gp.quicksum([v,l[s],d] for v in Vset) == x[l[s], d])